In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
import requests
import csv
import pandas as pd
import matplotlib.pyplot as plt
import nltk

pd.set_option('display.max_colwidth', None)

In [56]:
# Config

# Create Backup
# --------------------

os.system("mkdir backup")

0

In [ ]:
# Functions
#    []   scrapRichemont()                     #    []   scrapRichemont()
#    []   scrapDassaultAviation()              #    []   scrapRichemont()
#    []   scrapAirFrance()                     #    []   scrapRichemont()
#    []   scrapSanofi()                        #    []   scrapRichemont()
#    []   scrapHermes()                        #    []   scrapRichemont()
#    []   scrapFramatome()                     #    []   scrapRichemont()

In [4]:
# Initialize function
def scrapRichemont():
    richemont_job_db = []

    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')
    browser.get('https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow=1')

    soup = BeautifulSoup(browser.page_source,'lxml')

    for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
        row_scrap = []
        job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
        job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
        company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
        departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
        location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()

        row_scrap.append(job_title)
        row_scrap.append(job_link)
        row_scrap.append(company)
        row_scrap.append(departement)
        row_scrap.append(location)

        richemont_job_db.append(row_scrap)

    i = 14
    while i <= int(soup.find('a', class_='paginationItemLast')['href'][-3:]) :
        browser.get(f'https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow={i}')

        soup = BeautifulSoup(browser.page_source,'lxml')

        for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
            row_scrap = []
            job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
            job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
            company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
            departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
            location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(location)

            richemont_job_db.append(row_scrap)

        i = i + 14

    richemont_job_db = pd.DataFrame(richemont_job_db)
    richemont_job_db = richemont_job_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company",
                                                        3 : "Department", 4 : "Location"})

    richemont_job_db_copy = richemont_job_db
    richemont_job_db_copy['desc'] = ''

    # Add Desc to job offer
    # add https://jobs.richemont.com before links to connect

    i = 0

    while i <= len(richemont_job_db_copy)-1:
        browser.get('https://jobs.richemont.com' + richemont_job_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        richemont_job_db_copy['desc'][i] = soup.find('span', class_="jobdescription").text
        i = i + 1

    richemont_job_db_copy.to_csv('backup/richemont_db.csv')
    richemont_job_db_copy.to_json('backup/richemont_db.json')
    
    richemont_job_db_copy.head()
    
def scrapDassaultAviation():
    dassaultAviation_db = []

    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

    i = 1
    while i <= 10:
        browser.get(f'https://carriere.dassault-aviation.com/offre-de-emploi/liste-offres.aspx?page={i}&LCID=1036')
        soup = BeautifulSoup(browser.page_source,'lxml')

        for x in soup.find_all('li', class_='ts-offer-list-item'):
                row_scrap = []
                job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
                job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
                company = "Dassault Aviation"
                departement = 'N/A'
                details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

                row_scrap.append(job_title)
                row_scrap.append(job_link)
                row_scrap.append(company)
                row_scrap.append(departement)
                row_scrap.append(details)

                dassaultAviation_db.append(row_scrap)

        i = i + 1

    dassaultAviation_db = pd.DataFrame(dassaultAviation_db)
    dassaultAviation_db = dassaultAviation_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    dassaultAviation_db_copy = dassaultAviation_db
    dassaultAviation_db_copy['desc'] = ''

    # Add Desc to job offer
    # add https://carriere.dassault-aviation.com before links to connect

    i = 0
    while i <= len(dassaultAviation_db_copy)-1:
        browser.get('https://carriere.dassault-aviation.com' + dassaultAviation_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        dassaultAviation_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
        i = i + 1

    dassaultAviation_db_copy.to_csv('backup/dassaultAviation_db.csv')
    dassaultAviation_db_copy.head()
    
def scrapAirFrance():
    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

    airFrance_db = []

    browser.get('https://recrutement.airfrance.com/offre-de-emploi/liste-offres.aspx')
    soup = BeautifulSoup(browser.page_source,'lxml')

    for x in soup.find_all('li', class_='ts-offer-list-item'):
                row_scrap = []
                job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
                job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
                company = "Air France"
                departement = 'N/A'
                details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

                row_scrap.append(job_title)
                row_scrap.append(job_link)
                row_scrap.append(company)
                row_scrap.append(departement)
                row_scrap.append(details)

                airFrance_db.append(row_scrap)

    airFrance_db = pd.DataFrame(airFrance_db)
    airFrance_db = airFrance_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    airFrance_db_copy = airFrance_db
    airFrance_db_copy['desc'] = ''

    i = 0
    while i <= len(airFrance_db_copy)-1:
        browser.get('https://recrutement.airfrance.com' + airFrance_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        airFrance_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
        i = i + 1

    airFrance_db_copy.to_csv('backup/airFrance_db.csv')
    airFrance_db_copy.head()

def scrapSanofi():
    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')
    browser.get('https://fr.jobs.sanofi.com/recherche-d%27offres/?p=1')
    soup = BeautifulSoup(browser.page_source,'lxml')

    sanofi_db = []

    i = 1
    while i <= int(soup.find('input', class_='pagination-current')['max']):
        browser.get(f'https://fr.jobs.sanofi.com/recherche-d%27offres/?p={i}')
        soup = BeautifulSoup(browser.page_source,'lxml')
        for x in soup.find('ul', class_='unstyled-list job-list').find_all('li'):
                    row_scrap = []
                    job_title = x.find('h2', attrs={'class' : 'h3-style'}).text.replace('\n', '').strip()
                    job_link = x.find('a')['href']
                    company = "Sanofi"
                    departement = 'N/A'
                    details = x.find('span', attrs={'class' : 'job-location'}).text

                    row_scrap.append(job_title)
                    row_scrap.append(job_link)
                    row_scrap.append(company)
                    row_scrap.append(departement)
                    row_scrap.append(details)

                    sanofi_db.append(row_scrap)

        i = i + 1

    sanofi_db = pd.DataFrame(sanofi_db)
    sanofi_db = sanofi_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    sanofi_db_copy = sanofi_db
    sanofi_db_copy['desc'] = ''

    i = 0
    while i <= len(sanofi_db_copy)-1:
        browser.get('https://fr.jobs.sanofi.com/' + sanofi_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        try :
            sanofi_db_copy['desc'][i] = soup.find('div', attrs={'class' : 'ats-description'}).text
        except :
            sanofi_db_copy['desc'][i] = ''
        i = i + 1

    sanofi_db_copy.to_csv('backup/sanofi_db.csv')
    sanofi_db_copy.head()

def scrapHermes():
    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

    browser.get('https://talents.hermes.com/fr/')
    soup = BeautifulSoup(browser.page_source,'lxml')

    hermes_db = []

    for x in soup.find_all('div', class_='card-block'):
                    row_scrap = []
                    job_title = x.find('h2', attrs={'class' : 'offer-description bd-highlight block-with-text'}).text.replace('\n', '').strip()
                    job_link = x.find('a')['href']
                    company = "Hermes"
                    departement = 'N/A'
                    details = []
                    for v in x.find('ul', attrs={'class' : 'offer-list_info'}).find_all('li'): 
                        item = v.text
                        details.append(item)

                    #details.append(details_item)

                    row_scrap.append(job_title)
                    row_scrap.append(job_link)
                    row_scrap.append(company)
                    row_scrap.append(departement)
                    row_scrap.append(details)

                    hermes_db.append(row_scrap)

    hermes_db = pd.DataFrame(hermes_db)
    hermes_db = hermes_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    hermes_db_copy = hermes_db
    hermes_db_copy['desc'] = ''

    i = 0
    while i <= len(hermes_db_copy)-1:
        browser.get('https://talents.hermes.com/' + hermes_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        try :
            hermes_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'detailOffer'}).text
        except :
            hermes_db_copy['desc'][i] = ''
        i = i + 1

    hermes_db_copy.to_csv('backup/hermes_db.csv')
    hermes_db_copy.head()
    
def scrapFramatome():
    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')
    framatome_db = []

    i = 1
    while i <= 44:
        browser.get(f'https://framatome-career.talent-soft.com/offre-de-emploi/liste-offres.aspx?page={i}&LCID=1033')
        soup = BeautifulSoup(browser.page_source,'lxml')

        for x in soup.find_all('div', class_='ts-offer-card Layer'):
                        row_scrap = []
                        job_title = x.find('h3', attrs={'class' : 'ts-offer-card__title'}).text.replace('\n', '').strip()
                        job_link = x.find('h3', attrs={'class' : 'ts-offer-card__title'}).find('a')['href']
                        company = "Framatome"
                        departement = 'N/A'
                        details = []
                        for v in x.find('ul', attrs={'class' : 'ts-offer-card-content__list'}).find_all('li'): 
                            item = v.text
                            details.append(item)

                        row_scrap.append(job_title)
                        row_scrap.append(job_link)
                        row_scrap.append(company)
                        row_scrap.append(departement)
                        row_scrap.append(details)

                        framatome_db.append(row_scrap)
        i = i + 1

    framatome_db = pd.DataFrame(framatome_db)
    framatome_db = framatome_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    framatome_db_copy = framatome_db
    framatome_db_copy['desc'] = ''

    i = 0
    while i <= len(framatome_db_copy)-1:
        browser.get('https://framatome-career.talent-soft.com/' + framatome_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        try :
            framatome_db_copy['desc'][i] = soup.find('div', attrs={'class' : 'ts-offer-page__content'}).text
        except :
            framatome_db_copy['desc'][i] = ''
        i = i + 1

    framatome_db_copy.to_csv('backup/framatome_db.csv')
    framatome_db_copy.head()
    
def scrapEngie():
    browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

    engie_db = []

    i = 1
    while i <= 315:
        browser.get(f'https://jobs.engie.com/jobs/search/76073636/page{i}')
        soup = BeautifulSoup(browser.page_source,'lxml')
        for x in soup.find_all('div', class_='jlr_right_hldr'):
            row_scrap = []
            job_title = x.find('p').text.replace('\n', '').strip()
            job_link = x.find('a')['href']
            company = "Engie"
            departement = x.find('p', attrs={'class' : 'jlr_company'}).text.replace('\n', '').strip()
            details = []
            for v in x.find_all('p', attrs={'class' : 'jlr_cat_loc'}): 
                item = v.text.replace('\t', '').replace('\n', '')
                details.append(item)

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(details)

            engie_db.append(row_scrap)
        i = i + 1


    engie_db = pd.DataFrame(engie_db)
    engie_db = engie_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    engie_db_copy = engie_db
    engie_db_copy['desc'] = ''

    i = 0
    while i <= len(engie_db_copy)-1:
        browser.get(engie_db_copy['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        try :
            engie_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'description_box'}).text
        except :
            engie_db_copy['desc'][i] = ''
        i = i + 1

    engie_db_copy.to_csv('backup/engie_db.csv')
    engie_db_copy.head()

In [48]:
# ---------------
# Air Liquide Scrap
# ---------------
#    - Issue : redirect to homepage instead of career page

#browser.get('https://www.airliquide.com/fr/carrieres/offres-emploi')
#soup = BeautifulSoup(browser.page_source,'lxml')
#airLiquide_db = []
#for x in soup.find_all('tr'):
#    row_scrap = []
#    job_title = x.find('a').text.replace('\n', '').strip()
#    job_link = x.find('h3', attrs={'class' : 'ts-offer-card__title'}).find('a')['href']
#    company = "Framatome"
#    departement = 'N/A'
#    details = []
#    for v in x.find('ul', attrs={'class' : 'ts-offer-card-content__list'}).find_all('li'): 
#        item = v.text
#        details.append(item)

#    row_scrap.append(job_title)
#    row_scrap.append(job_link)
#    row_scrap.append(company)
#    row_scrap.append(departement)
#    row_scrap.append(details)

#    airLiquide_db.append(row_scrap)

#airLiquide_db = pd.DataFrame(airLiquide_db)
#airLiquide_db = airLiquide_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
#airLiquide_db

In [ ]:
## https://www.airliquide.com/fr/carrieres/offres-emploi
https://jobsearch.alstom.com/search/?createNewAlert=false&q=&locationsearch=
https://cc.wd3.myworkdayjobs.com/fr-FR/ChanelCareers
https://jobs.danone.com/search/?createNewAlert=false&q=&locationsearch=&optionsFacetsDD_country=FR&optionsFacetsDD_facility=&optionsFacetsDD_department=Experienced+professionals
https://careers.3ds.com/jobs?woc=%7B%22country%22%3A%5B%22country%2Ffrance%22%5D%7D&wocset=6
http://careers.disneylandparis.com/en/management-business/supply-chain-procurement
https://www.edf.fr/edf-recrute
https://jobs.engie.com
https://recrutement.fnacdarty.com/accueil.aspx?LCID=1036
https://www.place-emploi-public.gouv.fr
https://jobs.gecareers.com/global/en/search-results
https://www.invivo-group.com/fr/nos-offres
https://kering.wd3.myworkdayjobs.com/fr-FR/Kering?source=LinkedIn_Slots
https://careers.loreal.com/en_US/jobs/SearchJobs/
https://www.lisi-aerospace.com/en/join-us/careers/
https://www.lvmh.fr/talents/nous-rejoindre/nos-offres/liste-des-offres/?job=&place=&experience=&activity=&contract=&reference=#gt_offers-results
https://www.mbda-systems.com/jobs/?gestmax%5Bvac_sector%5D=&gestmax%5Bvac_localisation%5D=001&gestmax%5Bvac_job_type%5D=
https://jobs.moncler.com/search/?createNewAlert=false&q=&locationsearch=
https://motul-recrute.talent-soft.com/job/list-of-jobs.aspx
https://www.naval-group.com/fr/nous-rejoindre-85?keywords=&offerFamilyCategory=&contractType=&country=&city=&op=Rechercher&form_build_id=form-NzrJ7AsvKwvEKPwLDA2P5Qu6e3T6EKUpkTDoHGZS6Is&form_id=talent_soft_offers_filters_form#offer-list-content
https://nexter-recrutement.profils.org/accueil.aspx?LCID=1036
https://orange.jobs/jobs/search.do?keyword=
https://pernodricard.wd3.myworkdayjobs.com/fr-FR/pernod-ricard
https://jobs.groupe-psa.com/offre-de-emploi/liste-offres.aspx?mode=layer&lcid=1036&facet_JobDescription_Contract=577
https://renault.referrals.selectminds.com/
https://www.safran-group.com/jobs
https://joinus.saint-gobain.com/fr
https://www.sodern.com/website/fr/ref/Carrières_262.html
https://hris-suez.csod.com/ats/careersite/search.aspx?site=8&c=hris-suez&sid=%5e%5e%5eHJe5gko1mldbDMyZ8oI9Lw%3d%3d
https://careers.hr.technipfmc.com
https://emploi.thalesgroup.com/recherche-d%27offres
https://krb-sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=30080&siteid=6559#home
https://career012.successfactors.eu/career?company=VALLOUREC&site=VjItcmY2YVFFcnJMYWhIb3RmMzhTYU9Ldz09
https://emplois.vinci.com/recherche-d%27offres
https://www.nestle.fr/jobs/search-jobs?keyword=&country=FR&location=&career_area=All&company=All
https://www.smcp.com/fr/talents/offres-d-emploi/?keywords=&geographicalLocation=22&offerCountry=79&offerRegion=&organisation=&offerFamilyCategory=&contractType=&experienceLevel=
https://pfizer.wd1.myworkdayjobs.com/PfizerCareers/5/refreshFacet/318c8bb6f553100021d223d9780d30be
https://careers.faurecia.com/search/?createNewAlert=false&q=&locationsearch=france&optionsFacetsDD_customfield3=&optionsFacetsDD_country=&optionsFacetsDD_shifttype=Unlimited
https://www.emploi.sncf.com/nos-offres/contrat/577-578/localisation/40629/
https://careers.ratpdev.com/offre-de-emploi/liste-offres.aspx?page=3&LCID=1036
https://arianegroup.wd3.myworkdayjobs.com/EXTERNALALL